In [1]:
import torch
import torch.nn as nn #neral network
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import torchvision as tv
import torchvision.transforms as transforms

In [55]:
transform = transforms.Compose([tv.transforms.ToTensor(),
                                        tv.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
#what is batch norm difference from normalization
#fashionMNIST

In [56]:

# transform = transforms.Compose([tv.transforms.ToTensor(),

#             tv.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))]) 

trainset = tv.datasets.CIFAR10(root="data",train=True,download=False,transform=transform) 

In [57]:
trainset

Dataset CIFAR10
    Number of datapoints: 50000
    Split: train
    Root Location: data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None

In [58]:
#dataloader = torch.utils.data.DataLoader(trainset,batch_size=4, shuffle=False, num_workers=4)
dataloader = torch.utils.data.DataLoader(trainset,batch_size=4, shuffle=False, num_workers=4)

In [59]:
class OurModel(nn.Module):

    def __init__(self):

        super(OurModel,self).__init__()

        self.conv1 = nn.Conv2d(3,6,5) 
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [60]:
net = OurModel()

loss_func = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(),lr=0.001,weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [61]:
for epoch in range(2):

    running_loss= 0.0

    for i,data in enumerate(dataloader,0):

        inputs, labels = data
        optimizer.zero_grad()

        # forward prop

        outputs = net(inputs)

        loss = loss_func(outputs, labels)

        # backprop

        loss.backward() # compute gradients
        optimizer.step() # update parameters

        # print statistics

        running_loss += loss.item()

        if i %2000==1999: # print every 2000 mini-batches

            print('[epoch: %d, minibatch: %5d] loss: %.3f'%(epoch +1, i +1, running_loss /2000))

            running_loss = 0.0

print("Training finished!")

[epoch: 1, minibatch:  2000] loss: 2.187
[epoch: 1, minibatch:  4000] loss: 1.864
[epoch: 1, minibatch:  6000] loss: 1.657
[epoch: 1, minibatch:  8000] loss: 1.539
[epoch: 1, minibatch: 10000] loss: 1.517
[epoch: 1, minibatch: 12000] loss: 1.456
[epoch: 2, minibatch:  2000] loss: 1.399
[epoch: 2, minibatch:  4000] loss: 1.383
[epoch: 2, minibatch:  6000] loss: 1.332
[epoch: 2, minibatch:  8000] loss: 1.287
[epoch: 2, minibatch: 10000] loss: 1.305
[epoch: 2, minibatch: 12000] loss: 1.260
Training finished!


In [53]:
testset = tv.datasets.CIFAR10(root='data', train=False,
                                       download=True, transform=transform)

Files already downloaded and verified


In [54]:
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

dataiter = iter(testloader) 
images, labels = dataiter.next()

outputs = net(images)

_, predicted = torch.max(outputs, 1)

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4))) 
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

GroundTruth:    cat  ship  ship plane
Predicted:   bird  ship  ship plane
